In [1]:
# 실행 후 런타임 > 세션 다시시작 후 아래셀부터 실행하기
!pip install -U -q datasets transformers fsspec 
!pip install -q sentence-transformers faiss-cpu matplotlib seaborn pandas numpy tqdm scikit-learn nltk python-Levenshtein


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


1. **데이터셋 로드 및 준비**:
   - tatsu-lab/alpaca
   - yahma/alpaca-cleaned  
   - databricks/databricks-dolly-15k
   - 각 데이터셋의 구조 분석 및 통합 포맷으로 변환

3. **Semantic Deduplication 구현**  
   - Sentence-BERT 임베딩 생성 (`all-MiniLM-L6-v2` 등 경량 모델)  
   - FAISS 인덱싱 & k-NN 탐색  
   - Cosine 유사도 임계값(Threshold) 기반 중복 제거  
   - 제거된 샘플 & 유사도 예시 출력  

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
from tqdm import tqdm

from datasets import load_dataset, DatasetDict
import pandas as pd
import hashlib
import re
import Levenshtein
from sentence_transformers import SentenceTransformer
import faiss
import time
import tracemalloc

# 데이터셋 로드 및 구조 확인
ds1 = load_dataset("tatsu-lab/alpaca")
ds2 = load_dataset("yahma/alpaca-cleaned")
ds3 = load_dataset("databricks/databricks-dolly-15k")
# ds4 = load_dataset("timdettmers/openassistant-guanaco")

# 데이터셋 개수 확인
print("tatsu-lab/alpaca:", len(ds1["train"]))
print("yahma/alpaca-cleaned:", len(ds2["train"]))
print("databricks/databricks-dolly-15k:", len(ds3["train"]))

# 예시: train split만 살펴보기
print("─ tatsu-lab/alpaca sample ─")
print(ds1["train"][0])
print("\n─ yahma/alpaca-cleaned sample ─")
print(ds2["train"][0])
print("\n─ databricks/databricks-dolly-15k sample ─")
print(ds3["train"][0])
# print("\n─ openassistant-guanaco sample ─")
# print(ds4["train"][0])

model = SentenceTransformer('all-MiniLM-L6-v2')

tatsu-lab/alpaca: 52002
yahma/alpaca-cleaned: 51760
databricks/databricks-dolly-15k: 15011
─ tatsu-lab/alpaca sample ─
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

─ yahma/alpaca-cleaned sample ─
{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and health

In [ ]:
# 공통 포맷 변환 (instruction, input, output)
def unify(ds, cols_map):
    df = pd.DataFrame(ds)
    df = df.rename(columns=cols_map)[list(cols_map.values())]
    return df

df1 = unify(ds1["train"], {"instruction": "instruction", "input": "input", "output": "output"})
df2 = unify(ds2["train"], {"instruction": "instruction", "input": "input", "output": "output"})
df3 = unify(ds3["train"], {"instruction": "instruction",  "context": "input", "response": "output"})
# df4 = unify(ds4["train"], {"instruction": "instruction", "input": "input",   "output": "response"})

df = pd.concat([df1, df2, df3, ], ignore_index=True)
print("통합 데이터셋 크기:", df.shape)
print(df.head(3))


통합 데이터셋 크기: (118773, 3)
                            instruction input  \
0  Give three tips for staying healthy.         
1    What are the three primary colors?         
2    Describe the structure of an atom.         

                                              output  
0  1.Eat a balanced diet and make sure to include...  
1  The three primary colors are red, blue, and ye...  
2  An atom is made up of a nucleus, which contain...  


In [7]:
#  임베딩 생성
model = SentenceTransformer("all-MiniLM-L6-v2")  # ~22M 파라미터
texts = df["instruction"].tolist()

print("Embedding 생성 시작...")
embs = model.encode(texts, batch_size=128, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
print("✅ 임베딩 생성 완료:", embs.shape)


Embedding 생성 시작...


Batches:   0%|          | 0/928 [00:00<?, ?it/s]

✅ 임베딩 생성 완료: (118773, 384)


In [8]:
# FAISS 인덱스 구축 (Inner Product → 코사인 유사도)
d = embs.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embs)
print("✅ FAISS 인덱스에 추가 완료. 총 벡터 수:", index.ntotal)


✅ FAISS 인덱스에 추가 완료. 총 벡터 수: 118773


In [9]:
# Deduplication 파라미터 설정 & 수행
threshold = 0.85  # 코사인 유사도 임계값 (0.0 ~ 1.0)
k = 2            # 각 벡터당 자기 자신 + 가장 유사한 1개

# 모든 벡터에 대해 k-NN 검색
D, I = index.search(embs, k)
# D: inner product 결과. cosine similarity = inner product (embs normalized)
# I: 인덱스

to_remove = set()
for i, (sims, nbrs) in enumerate(zip(D, I)):
    sim, nbr = sims[1], nbrs[1]
    if sim >= threshold:
        # i와 nbr 중 인덱스가 큰 쪽만 제거(keep order)
        remove_idx = max(i, nbr)
        to_remove.add(remove_idx)

before = len(df)
after = before - len(to_remove)
print(f"⚙️ Deduplication 수행: threshold={threshold}")
print(f"중복 제거 전: {before} → 후: {after}  (제거: {len(to_remove)})")

⚙️ Deduplication 수행: threshold=0.85
중복 제거 전: 118773 → 후: 28227  (제거: 90546)


In [13]:
# 결과 DataFrame 생성 & 예시 출력
keep_idx = [i for i in range(len(df)) if i not in to_remove]
df_dedup = df.iloc[keep_idx].reset_index(drop=True)

before=len(df)
after=len(df_dedup)

print(f"중복 제거 전 {before} → 제거 후 {after}")
print(f"제거 sample 수: {before - after}")
print(f"제거 sample 비율: {(before - after) / before * 100:.2f}%")

print("\n🔍 유사도가 높은 샘플 예시:")
count = 0
k=3
for i in list(to_remove)[:k]:
    # 제거된 중복데이터
    sim, nbr = D[i][1], I[i][1]
    print(f"\n샘플 인덱스 {nbr} ↔ 제거 인덱스 {i} (cos_sim={sim:.3f})")
    print("▶ 원본 instruction:", df.loc[nbr, "instruction"])
    print("▶ 제거된 instruction:", df.loc[i, "instruction"])
    count += 1
    if count >= k:
        break

중복제거 데이터 shape: (28227, 3)

🔍 유사도가 높은 샘플 예시:

샘플 인덱스 0 ↔ 제거 인덱스 0 (cos_sim=1.000)
▶ 원본 instruction: Give three tips for staying healthy.
▶ 제거된 instruction: Give three tips for staying healthy.

샘플 인덱스 1 ↔ 제거 인덱스 1 (cos_sim=1.000)
▶ 원본 instruction: What are the three primary colors?
▶ 제거된 instruction: What are the three primary colors?

샘플 인덱스 3 ↔ 제거 인덱스 3 (cos_sim=1.000)
▶ 원본 instruction: How can we reduce air pollution?
▶ 제거된 instruction: How can we reduce air pollution?


In [15]:
# 1) to_remove 리스트로 변환하고 (idx, sim) 튜플 생성
remove_sims = [(i, D[i][1]) for i in to_remove]

# 2) sim 오름차순 정렬
remove_sims_sorted = sorted(remove_sims, key=lambda x: x[1])

# 3) 가장 sim이 낮은 2개 선택
k=10
lowest = remove_sims_sorted[:k]

print("\n🔍 유사도(코사인) 가장 낮은 제거:")
for rank, (rem_idx, sim) in enumerate(lowest, 1):
    nbr_idx = I[rem_idx][1]  # rem_idx와 짝인 neighbor
    print(f"\n페어 {rank}: (cos_sim = {sim:.3f})")
    print("▶ 원본 instruction:", df.loc[nbr_idx, "instruction"])
    print("▶ 제거된 instruction:", df.loc[rem_idx,  "instruction"])
    print("---")


🔍 유사도(코사인) 가장 낮은 제거:

페어 1: (cos_sim = 0.850)
▶ 원본 instruction: Design a motto for the given company.
▶ 제거된 instruction: Create a motto for the given firm.
---

페어 2: (cos_sim = 0.850)
▶ 원본 instruction: Name two famous Asian celebrities.
▶ 제거된 instruction: Who are the most influential Asian celebrities?
---

페어 3: (cos_sim = 0.850)
▶ 원본 instruction: Identify which instrument is string or percussion: Kakko, Akonting
▶ 제거된 instruction: Identify which instrument is string or percussion: Wood block, Akkordolia
---

페어 4: (cos_sim = 0.850)
▶ 원본 instruction: Simplify this math equation
▶ 제거된 instruction: Divide the equation below and simplify.
---

페어 5: (cos_sim = 0.850)
▶ 원본 instruction: What is the best tv series in the world
▶ 제거된 instruction: 3 Best TV series in the World
---

페어 6: (cos_sim = 0.850)
▶ 원본 instruction: Recommend a way of studying for a test.
▶ 제거된 instruction: What's the best way to study for a test?
---

페어 7: (cos_sim = 0.850)
▶ 원본 instruction: How does the federal go

Tips:
- threshold 값을 0.8~0.9 사이에서 조정하며“유사 문장”제거 강도 조절
- 대규모 데이터셋에서는 FAISS GPU Index나 batch 처리 사용으로 속도향상